## Querying the EFD

Craig Lage 23-Sep-24

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')

## Available EFD clients

In [ ]:
EfdClient.list_efd_names()

## List EFD topics

In [ ]:
topics = await client.get_topics()
topics

## Look for specific EFD topics

In [ ]:
topics = await client.get_topics()
for topic in topics:
    if 'Hexapod' in topic:
        print(topic)

## Query a specific topic

In [ ]:
start = Time("2022-05-01T00:00:00", scale='utc')
end = Time("2023-02-01T00:00:00", scale='utc')
vac = await client.select_time_series("lsst.sal.ATCamera.focal_plane_Reb", \
                                            ['temp20'], \
                                             start, end)


In [ ]:
plt.title("WREB temp20 Temperatures with no control")
plt.ylabel("Temperature(C)")
vac['temp20'].plot()
plt.savefig(f"/home/c/cslage/u/AuxTel/temperatures/WREB_Uncontrolled.pdf")

In [ ]:
start = Time("2024-05-01T00:00:00", scale='utc')
end = Time("2024-10-15T00:00:00", scale='utc')
vac1 = await client.select_time_series("lsst.sal.ATCamera.focal_plane_Reb", \
                                            ['temp20'], \
                                             start, end)
start = Time("2024-12-15T00:00:00", scale='utc')
end = Time("2025-01-23T00:00:00", scale='utc')
vac2 = await client.select_time_series("lsst.sal.ATCamera.focal_plane_Reb", \
                                            ['temp20'], \
                                             start, end)


In [ ]:
vac3 = pd.concat([vac1, vac2])

In [ ]:
plt.title("WREB temp20 Temperatures with control")
plt.ylabel("Temperature(C)")
vac3['temp20'].plot()
plt.savefig(f"/home/c/cslage/u/AuxTel/temperatures/WREB_Controlled.pdf")

## List the data in the topic

In [ ]:
el.columns

## Plot a data column

In [ ]:
el['actualPosition'].plot()

## Another example

In [ ]:
start = Time("2024-09-16 19:45:00Z", scale='utc')
end = Time("2024-09-16 20:00:00Z", scale='utc')
gill = await client.select_time_series("lsst.sal.ESS.airTurbulence", \
                                            ["*"],  start, end, index=205)
len(gill)
print(gill.columns)
gill['sonicTemperatureStdDev'].plot()

In [ ]:
start = Time("2024-09-15 12:00:00Z", scale='utc')
end = Time("2024-09-17 00:00:00Z", scale='utc')
camp = await client.select_time_series("lsst.sal.ESS.airTurbulence", \
                                            ["*"],  start, end, index=201)
print(len(camp))
#print(camp.columns)
camp['sonicTemperatureStdDev'].plot()
print(camp.tail(1))

## Packed data example

In [ ]:
start = Time("2024-09-26T03:00:00", scale='utc')
end = Time("2024-09-26T07:00:00", scale='utc')
vel = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", \
                                            ['azimuthMotor1Velocity', 'azimuthMotor2Velocity'], \
                                             start, end)


In [ ]:
plt.title("Observing night - 20240925")
vel['azimuthMotor2Velocity'].plot()
plt.ylabel("Az Velocity (deg/sec)")
plt.savefig(f"/home/c/cslage/u/AuxTel/broken_limit_switch/Typical_Slew_Speeds.pdf")

In [ ]:
start = Time("2024-10-01T13:00:00", scale='utc')
end = Time("2024-10-01T19:00:00", scale='utc')
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            ['azimuthCalculatedAngle'], \
                                             start, end)
cw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCW", \
                                            ['*'], \
                                             start, end)
ccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCCW", \
                                            ['*'], \
                                             start, end)
limcw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCW", \
                                            ['*'], \
                                             start, end)
limccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCCW", \
                                            ['*'], \
                                             start, end)
print(len(az), len(cw), len(ccw), len(limcw), len(limccw))

In [ ]:
fig, ax = plt.subplots(1,1)
az['azimuthCalculatedAngle'].plot(ax=ax)
if len(cw) > 0:
    for time in cw[cw['active']==True].index:
        ax.axvline(time, ls='--', color='black')
if len(ccw) > 0:
    for time in ccw[ccw['active']==True].index:
        ax.axvline(time, ls='--', color='red')


In [ ]:
start = Time("2024-09-26T15:00:00", scale='utc')
end = Time("2024-09-26T16:45:00", scale='utc')
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            ['azimuthCalculatedAngle'], \
                                             start, end)
cw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCW", \
                                            ['*'], \
                                             start, end)
ccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCCW", \
                                            ['*'], \
                                             start, end)
limcw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCW", \
                                            ['*'], \
                                             start, end)
limccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCCW", \
                                            ['*'], \
                                             start, end)
print(len(cw), len(ccw), len(limcw), len(limccw))
vel = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", \
                                            ['azimuthMotor1Velocity', 'azimuthMotor2Velocity'], \
                                             start, end)

In [ ]:
fig, axs = plt.subplots(2,1)
plt.subplots_adjust(hspace=0.7)
plt.suptitle("Incident - 20240926", fontsize=18)
az['azimuthCalculatedAngle'].plot(ax=axs[0])
for time in cw[cw['active']==True].index:
    axs[0].axvline(time, ls='--', color='black', label="CW Topple")
labeled = False
for time in ccw[ccw['active']==True].index:
    if not labeled:
        axs[0].axvline(time, ls='--', color='red', label="CCW Topple")
        labeled = True
    else:
        axs[0].axvline(time, ls='--', color='red', label='')
for time in limcw[limcw['active']==True].index:
    axs[0].axvline(time, ls='--', color='orange', label="Limit switch")
vel['azimuthMotor2Velocity'].plot(ax=axs[1])
axs[0].set_ylabel("Azimuth (degrees)")
axs[0].legend(loc='upper left')
axs[0].set_ylim(-180, 360)
axs[1].set_ylabel("Az velocity (deg/sec)")

plt.savefig(f"/home/c/cslage/u/AuxTel/broken_limit_switch/Incident_26Sep24.pdf")

In [ ]:
start = Time("2024-09-26T16:31:30", scale='utc')
end = Time("2024-09-26T16:32:30", scale='utc')
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            ['azimuthCalculatedAngle'], \
                                             start, end)
cw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCW", \
                                            ['*'], \
                                             start, end)
ccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCCW", \
                                            ['*'], \
                                             start, end)
limcw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCW", \
                                            ['*'], \
                                             start, end)
limccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCCW", \
                                            ['*'], \
                                             start, end)
print(len(cw), len(ccw), len(limcw), len(limccw))
vel = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", \
                                            ['azimuthMotor1Velocity', 'azimuthMotor2Velocity'], \
                                             start, end)

In [ ]:
fig, axs = plt.subplots(2,1)
plt.subplots_adjust(hspace=0.5)
plt.suptitle("Incident Blowup 1 - 20240926", fontsize=18)
az['azimuthCalculatedAngle'].plot(ax=axs[0])
axs[0].text(az.index[0], 100, f"Max Az = {np.max(az['azimuthCalculatedAngle'].values):.1f} degrees")
#for time in cw[cw['active']==True].index:
#    ax.axvline(time, ls='--', color='black')
#for time in ccw[ccw['active']==True].index:
#    ax.axvline(time, ls='--', color='red')
for time in limcw[limcw['active']==True].index:
    axs[0].axvline(time, ls='--', color='orange', label='Limit switch')
axs[0].legend()
vel['azimuthMotor2Velocity'].plot(ax=axs[1])
axs[1].text(az.index[0], 10, f"Max Vel = {np.max(vel['azimuthMotor2Velocity'].values):.1f} deg/sec")
plt.savefig(f"/home/c/cslage/u/AuxTel/broken_limit_switch/Incident_Blowup_1_26Sep24.pdf")

In [ ]:
start = Time("2024-09-26T16:32:00", scale='utc')
end = Time("2024-09-26T16:32:03", scale='utc')
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            ['azimuthCalculatedAngle'], \
                                             start, end)
cw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCW", \
                                            ['*'], \
                                             start, end)
ccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCCW", \
                                            ['*'], \
                                             start, end)
limcw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCW", \
                                            ['*'], \
                                             start, end)
limccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCCW", \
                                            ['*'], \
                                             start, end)
print(len(cw), len(ccw), len(limcw), len(limccw))
vel = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", \
                                            ['azimuthMotor1Velocity', 'azimuthMotor2Velocity'], \
                                             start, end)

In [ ]:
fig, axs = plt.subplots(2,1)
plt.subplots_adjust(hspace=0.5)
plt.suptitle("Incident Blowup 2 - 20240926", fontsize=18)
t0 = Time(az.index[0]).unix_tai
times = Time(az.index[:]).unix_tai - t0
axs[0].plot(times, az['azimuthCalculatedAngle'].values)
axs[0].set_ylim(250, 300)
axs[0].axhline(265.0, ls='--', color='black', label = "Software limit??")

axs[0].set_ylabel("Azimuth (degrees)")
axs[0].set_xlabel("Time(seconds)")
for time in limcw[limcw['active']==True].index:
    thisTime = Time(time).unix_tai - t0
    axs[0].axvline(thisTime, ls='--', color='orange', label="Limit switch")
axs[0].legend()
axs[1].plot(times, vel['azimuthMotor2Velocity'].values)
axs[1].set_ylabel("Az velocity (deg/se)")
axs[1].set_xlabel("Time(seconds)")
axs[1].text(0.2, 5, f"Max Vel = {np.max(vel['azimuthMotor2Velocity'].values):.1f} deg/sec")
plt.savefig(f"/home/c/cslage/u/AuxTel/broken_limit_switch/Incident_Blowup_2B_26Sep24.pdf")

In [ ]:
#Others to check

'lsst.sal.ATMCS.logevent_azimuthLimitSwitchCCW',
'lsst.sal.ATMCS.logevent_azimuthLimitSwitchCW',
'lsst.sal.ATMCS.logevent_azimuthToppleBlockCCW',
'lsst.sal.ATMCS.logevent_azimuthToppleBlockCW',

In [ ]:
#start = Time("2024-09-10T02:30:00", scale='utc') # ccw topple
#end = Time("2024-09-10T02:45:00", scale='utc') # ccw topple
start = Time("2024-09-10T15:02:00", scale='utc')
end = Time("2024-09-10T15:07:00", scale='utc')

az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            ['azimuthCalculatedAngle'], \
                                             start, end)
cw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCW", \
                                            ['*'], \
                                             start, end)
ccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCCW", \
                                            ['*'], \
                                             start, end)
limcw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCW", \
                                            ['*'], \
                                             start, end)
limccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCCW", \
                                            ['*'], \
                                             start, end)
print(len(cw), len(ccw), len(limcw), len(limccw))

In [ ]:
fig, ax = plt.subplots(1,1)
az['azimuthCalculatedAngle'].plot(ax=ax)
for time in cw[cw['active']==True].index:
    ax.axvline(time, ls='--', color='black')
for time in ccw[ccw['active']==True].index:
    ax.axvline(time, ls='--', color='red')



In [ ]:
start = Time("2024-09-10T02:42:00", scale='utc') # ccw topple
end = Time("2024-09-10T02:47:00", scale='utc') # ccw topple
#start = Time("2024-09-10T15:02:00", scale='utc')
#end = Time("2024-09-10T15:07:00", scale='utc')

az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            ['azimuthCalculatedAngle'], \
                                             start, end)
cw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCW", \
                                            ['*'], \
                                             start, end)
ccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthToppleBlockCCW", \
                                            ['*'], \
                                             start, end)
limcw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCW", \
                                            ['*'], \
                                             start, end)
limccw = await client.select_time_series("lsst.sal.ATMCS.logevent_azimuthLimitSwitchCCW", \
                                            ['*'], \
                                             start, end)
print(len(cw), len(ccw), len(limcw), len(limccw))

In [ ]:
fig, ax = plt.subplots(1,1)
az['azimuthCalculatedAngle'].plot(ax=ax)
for time in cw[cw['active']==True].index:
    ax.axvline(time, ls='--', color='black')
for time in ccw[ccw['active']==True].index:
    ax.axvline(time, ls='--', color='red')



In [ ]:
start = Time("2024-09-25T00:00:00", scale='utc')
end = Time("2024-09-30T23:00:00", scale='utc')
fe80 = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                            ['secondaryCylinderFollowingError80'], \
                                             start, end)


In [ ]:
fe80['secondaryCylinderFollowingError80'].plot()
pstart = Time("2024-09-30T21:00:00", scale='utc').isot
pend = Time("2024-09-30T23:00:00", scale='utc').isot
plt.xlim(pstart,pend)
plt.ylim(-10,10)

In [ ]:
start = Time("2024-09-04T21:00:00", scale='utc')
end = Time("2024-09-04T21:01:00", scale='utc')
fe80 = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                            ['secondaryCylinderFollowingError80'], \
                                             start, end)
fe80['secondaryCylinderFollowingError80'].plot()
plt.ylim(-10,10)


In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
pdf = PdfPages('/home/c/cslage/u/MTM1M3/data/Test_330.pdf')
fig = plt.figure(figsize=(5,5))
for day in range(1, 31):
    for hour in range(24):
        try:
            start = Time(f"2024-09-{day:02d}T{hour:02d}:00:00", scale='utc')
            end = Time(f"2024-09-{day:02d}T{hour:02d}:01:00", scale='utc')
            fe80 = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                        ['secondaryCylinderFollowingError80'], \
                                                         start, end)
            fe80['secondaryCylinderFollowingError80'].plot()
            plt.title(f"FA330 Secondary Following Error 2024-09-{day}T{hour}:00")
            plt.ylim(-10,10)
            pdf.savefig(fig)  # saves the current figure into a pdf page
            plt.clf()
        except:
            continue
    print(f"Finished day {day}")
pdf.close()



In [ ]:
start = Time("2024-10-07T12:00:00", scale='utc')
end = Time("2024-10-07T15:00:00", scale='utc')
acc = await client.select_time_series("lsst.sal.MTM1M3.accelerometerData", \
                                            ['*'], \
                                             start, end)


In [ ]:
acc.head(2)

In [ ]:
acc.tail(2)

In [ ]:
acc.columns

In [ ]:
acc['accelerometer0'].plot()